In [ ]:
# NESTEROV ACCELERATED GRADIENT DESCENT

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import seaborn as sns

load_data = pd.read_excel(r"C:\Users\Krishnasai\Load Data in kW (1).xlsx")
load_data.head()

type(load_data)

data = load_data.rename(columns={5551.82208:'load'},inplace=False)

data.head()

data.shape

data.describe()

data.info()

data.corr()

# VALIDATING NULL VALUE

data.isnull().sum()

# VISUALISING OUTLIERS OF DATA

sns.boxplot(x=data['load'])

from scipy import stats

z = np.abs(stats.zscore(load_data.iloc[:,0]))

threshold = 3
print(np.where(z > 3))

#From boxplot and from above zscore calculation, we can conclude that there is one outlier. lets replace it.

data.iloc[928,0] = data.iloc[927,0]

#AFTER REPLACING THE OUTLIER

sns.boxplot(x=data['load'])

#now all outliers are removed and now we are good to train the model

data_x = []
data_y = []
for i in range(0,len(data)-1):
  data_x.append(data.iloc[i,0])
  data_y.append(data.iloc[i+1,0])

plt.scatter(data_x,data_y)
plt.title("load data visualisation")
plt.xlabel("previous hours load")
plt.ylabel("present hours load")
plt.grid()

data_xx = pd.DataFrame(data_x)
min_x = float(data_xx.min())
max_x = float(data_xx.max())
data_yy = pd.DataFrame(data_y)
min_y = float(data_yy.min())
max_y = float(data_yy.max())
print(min_x)
print(min_y)
print(max_x)
print(max_y)

# DATA NORMALISATION

x = minmax_scale(data_x)
y = minmax_scale(data_y)

data_norm = []
for i in range(len(data)-24):
    data_norm.append([x[i],y[i]])

data_norm = pd.DataFrame(data_norm)
data_norm

# SPLITTING DATA SET

from sklearn.model_selection import train_test_split
data_train,data_test = train_test_split(data_norm,test_size=0.1)

data_norm.head()

data_x = np.array(data_train[0])

data_y = np.array(data_train[1])

len(data_x)

len(data_test)

# TRAINING THE MODEL

#STEP 2: initialising required parameters
m = 1
c = -1
iter = 1
epochs = 2000
alpha = 0.1
ns = len(data_train)
error = []
vm = 0
vc = 0
moment = 0.9

# while loop is runned until iter reaches epochs
while (iter<=epochs):
  # running for loop to calculate and update model parameters for each sample
  for i in range(0,len(data_train)):
    # calculating derivatives of m and c
    der_m = (-1) * (data_y[i] - ((m+(moment*vm))*data_x[i]) - (c+(moment*vc)))*(data_x[i])
    der_c = (-1) * (data_y[i] - ((m+(moment*vm))*data_x[i]) - (c+(moment*vc)))
    # print("derivatives at ",i,"   ",der_m,der_c)

    # calculating change in m and c
    vm = (moment * vm) - (alpha * der_m)
    vc = (moment * vc) - (alpha * der_c)
    # print("changes at ",i,vm,vc)

    #updating m and c
    m = m + vm
    c = c + vc
    # print("values at",i,m,c)

  err=0
  for i in range(0,len(data_train)):
    err+=((data_y[i] - (m * data_x[i]) - c))**2
  err = (1/(2*len(data_train))) * err
  error.append(err)
    #incrementing no of iterations
  iter = iter + 1
 

  
print(m,c)

len(error)

iters = np.arange(epochs)

from sklearn.metrics import mean_squared_error

pred = []
for i in range(len(data_train)):
  pred.append(m*data_x[i] + c)

mse_train = mean_squared_error(data_y,pred)

data_test

pred_test = m*data_test[0] + c
mse_test = mean_squared_error(data_test[1],pred_test)

# MEAN SQUARE ERRORS

print("training data mean squared error:",mse_train)
print("Testing data mean squared error:",mse_test)

print("model parameter:",m,c)

#VISUALISING TRAINING DATA

plt.scatter(data_train[0],data_train[1],label="original data points")
plt.plot(data_train[0],pred,color='red',label="predicted line")
plt.title("visualing regression line of training data")
plt.xlabel("previous hours load")
plt.ylabel("present hours load")
plt.grid()
plt.legend()

#VISUALISING TESTING DATA

plt.scatter(data_test[0],data_test[1],label="original data points")
plt.plot(data_test[0],pred_test,color='red',label="predicted line")
plt.title("visualing regression line of testing data")
plt.xlabel("previous hours load")
plt.ylabel("present hours load")
plt.grid()
plt.legend()

#DEPLOYMENT OF MODEL

input_load = float(input("enter yesterday load at this time : "))
input_load = (input_load - min_x)/(max_x-min_x)
output_load = input_load*m+c

output_load = output_load*(max_y-min_y)+min_y

print("predicted load for this hour is : ",output_load)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2183 entries, 0 to 2182
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   load    2183 non-null   float64
dtypes: float64(1)
memory usage: 17.2 KB
(array([928], dtype=int64),)
3377.9196
3377.9196
8841.66948
8841.66948
0.8169544215005555 0.13719107428858718
training data mean squared error: 0.016822233203693252
Testing data mean squared error: 0.019586687219796544
model parameter: 0.8169544215005555 0.13719107428858718
